In [1]:
import mlflow
import pandas as pd
import numpy as np
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [20]:
df = pd.read_csv("IMDB.csv")
df = df.sample(500)
df.to_csv("data.csv", index=False)
df.head()

,review,sentiment
775,A labor of love. Each frame is picture perfect...,positive
27,I can't get over how lazy the director is with...,negative
846,"This movie should be called ""plan 9 from josep...",negative
101,"What can I say? I think I have to write ""Spoil...",negative
357,This film was the first British teen movie to ...,positive


In [21]:
len(pd.read_csv("data.csv"))

500

In [22]:
df["sentiment"].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [23]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [24]:
df = normalize_text(df) 
df.head()

,review,sentiment
775,labor love frame picture perfect grab you shee...,positive
27,can t get lazy director movie instead setting ...,negative
846,movie called plan joseph smith think weirdness...,negative
101,say think write spoiler alert reveal used f wo...,negative
357,film first british teen movie actually address...,positive


In [26]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df.head()

,review,sentiment
775,labor love frame picture perfect grab you shee...,1
27,can t get lazy director movie instead setting ...,0
846,movie called plan joseph smith think weirdness...,0
101,say think write spoiler alert reveal used f wo...,0
357,film first british teen movie actually address...,1


In [27]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [34]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [30]:
import dagshub
mlflow.set_tracking_uri("https://dagshub.com/keshav1017/Capstone-Project.mlflow")
dagshub.init(repo_owner='keshav1017', repo_name='Capstone-Project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\ASUS\miniconda3\envs\atlas\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8e3e7fc0-12a7-401a-bcd0-2516a7023513&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=4d65d9fab1b9e910b2f804b677692445302c83d1669bcfd438ff0cdb2d320228




Accessing as keshav1017

Initialized MLflow to track repo "keshav1017/Capstone-Project"

Repository keshav1017/Capstone-Project initialized!

2025/06/17 16:55:34 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e5e8d96a326a4b309499f460435799e1', creation_time=1750159535809, experiment_id='0', last_update_time=1750159535809, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [36]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting Mlflow run...")

with mlflow.start_run():
    start_time = time.time()

    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Inititalizing Logistic Regression...")
        model = LogisticRegression(max_iter=1000) # increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, name="model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")
    except Exception as e:
        logging.error(f"An error occured: {e}", exc_info=True)

2025-06-17 17:18:09,717 - INFO - Starting Mlflow run...
2025-06-17 17:18:09,787 - WARNING - Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /keshav1017/Capstone-Project.mlflow/api/2.0/mlflow/runs/create
2025-06-17 17:18:10,372 - INFO - Logging preprocessing parameters...
2025-06-17 17:18:11,709 - INFO - Inititalizing Logistic Regression...
2025-06-17 17:18:11,711 - INFO - Fitting the model...
2025-06-17 17:18:11,760 - INFO - Model training complete.
2025-06-17 17:18:11,762 - INFO - Making predictions...
2025-06-17 17:18:11,765 - INFO - Calculating evaluation metrics...
2025-06-17 17:18:11,784 - INFO - Logging evaluation metrics...
2025-06-17 17:18:16,928 - INFO - Saving and logging the model...
2025-06-17 17:18:17,779 - ERROR - An error occured: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@d

🏃 View run thoughtful-trout-793 at: https://dagshub.com/keshav1017/Capstone-Project.mlflow/#/experiments/0/runs/e09178b1c24c475bbb53b9e3f8ae3cc4
🧪 View experiment at: https://dagshub.com/keshav1017/Capstone-Project.mlflow/#/experiments/0
